# New York Working Families Tax Credit Act

https://www.nysenate.gov/legislation/bills/2021/S9610

* Limits NY EITC to childless filers
* Replaces NY CTC with a new one that’s \$1,500/child, phasing down to \$500 at a 2% rate above \$25k (non-joint) or \$50k (joint)
* Pays quarterly


In [1]:
from policyengine_us import Microsimulation, Simulation
from policyengine_core.reforms import Reform
from policyengine_us.model_api import *

# Define the reform.
MAX_AMOUNT = 1_500
MIN_AMOUNT = 500
MTR = 0.02
THRESHOLD_JOINT = 50_000
THRESHOLD_NON_JOINT = 25_000

# Limit NY EITC to childless filers.
class ny_eitc(Variable):
    value_type = float
    entity = TaxUnit
    label = "NY EITC"
    unit = USD
    definition_period = YEAR
    reference = "https://www.nysenate.gov/legislation/laws/TAX/606"  # (d)
    defined_for = StateCode.NY

    def formula(tax_unit, period, parameters):
        eitc = tax_unit("earned_income_tax_credit", period)
        rate = parameters(period).gov.states.ny.tax.income.credits.eitc.match
        tentative_nys_eitc = eitc * rate
        household_credit = tax_unit("ny_household_credit", period)
        eligible = tax_unit("eitc_child_count", period) == 0
        return eligible * max_(0, tentative_nys_eitc - household_credit)


class ny_ctc(Variable):
    value_type = float
    entity = TaxUnit
    label = "NY CTC"
    documentation = "New York's Empire State Child Credit"
    unit = USD
    definition_period = YEAR
    reference = "https://www.nysenate.gov/legislation/laws/TAX/606"  # (c-1)
    defined_for = StateCode.NY

    def formula(tax_unit, period, parameters):
        ny_agi = tax_unit("ny_agi", period)
        eligible_children = tax_unit("eitc_child_count", period)
        max_amount = MAX_AMOUNT * eligible_children
        min_amount = MIN_AMOUNT * eligible_children
        filing_status = tax_unit("filing_status", period)
        threshold = where(
            filing_status == filing_status.possible_values.JOINT,
            THRESHOLD_JOINT,
            THRESHOLD_NON_JOINT,
        )
        excess = max_(0, ny_agi - threshold)
        reduction = excess * MTR
        return max_(max_amount - reduction, min_amount)


class implement_ny_wftc(Reform):
    def apply(self):
        self.update_variable(ny_eitc)
        self.update_variable(ny_ctc)


baseline = Microsimulation()
reformed = Microsimulation(reform=implement_ny_wftc)

(
    reformed.calc("ny_income_tax").sum() / 1e9
    - baseline.calc("ny_income_tax").sum() / 1e9
)


-3.10809176350293

In [2]:
import pandas as pd

YEAR = 2023


def make_hh(adults, children, child_age):
    people = dict(head=dict(age=30))
    members = ["head"]
    if adults == 2:
        people["spouse"] = dict(age=30)
        members += ["spouse"]
    for i in range(children):
        people[f"child{i}"] = dict(age=child_age)
        members += [f"child{i}"]
    return dict(
        people=people,
        households=dict(household=dict(members=members, state_code="NY")),
        axes=[[dict(name="employment_income", count=201, min=0, max=200_000)]],
    )


l = []
for adults in [1, 2]:
    for children in [0, 1, 2, 3]:
        for child_age in [1, 5]:
            hh = make_hh(adults, children, child_age)
            baseline_hh = Simulation(situation=hh)
            reformed_hh = Simulation(situation=hh, reform=implement_ny_wftc)
            l.append(
                pd.DataFrame(
                    dict(
                        adults=adults,
                        children=children,
                        child_age=child_age,
                        # Reshape combined array to get head's varied earnings.
                        earnings=baseline_hh.calculate(
                            "employment_income", YEAR
                        )[0 :: (adults + children)],
                        baseline_net_income=baseline_hh.calculate(
                            "spm_unit_net_income", YEAR
                        ),
                        reformed_net_income=reformed_hh.calculate(
                            "spm_unit_net_income", YEAR
                        ),
                        baseline_ny_ctc=baseline_hh.calculate("ny_ctc", YEAR),
                        reformed_ny_ctc=reformed_hh.calculate("ny_ctc", YEAR),
                        baseline_ny_eitc=baseline_hh.calculate(
                            "ny_eitc", YEAR
                        ),
                        reformed_ny_eitc=reformed_hh.calculate(
                            "ny_eitc", YEAR
                        ),
                    )
                )
            )

df = pd.concat(l)
df["net_gain"] = df["reformed_net_income"] - df["baseline_net_income"]


In [3]:
import plotly.express as px

px.line(
    df,
    "earnings",
    "net_gain",
    color="children",
    facet_col="adults",
    facet_row="child_age",
).update_layout(title="Effect of NY WFTC on net income")


In [4]:
import plotly.express as px

px.line(
    df,
    "earnings",
    "reformed_ny_ctc",
    color="children",
    facet_col="adults",
    facet_row="child_age",
).update_layout(title="Proposed NY CTC")


In [5]:
import plotly.express as px

px.line(
    df,
    "earnings",
    "reformed_ny_eitc",
    color="children",
    facet_col="adults",
    facet_row="child_age",
).update_layout(title="Proposed NY EITC")
